In [34]:
import pandas as pd
import glob
import os

In [2]:
path_to_perfiles = "10_APC_PERFILES"

# What does this workbook do?

### - Read all APC Propeller performance files (PER3_xxxx.dat) in a specific folder
### - add ISO units
### - store ALL data in ONE database



### Define my functions

In [38]:
def get_diam_pitch_ext(filename):
    
    #print(filename)
    file = filename.split("_")[1].split(".")[0] 

    diam,pitch = file.split("x")


    ##### get diameter
    if len(diam)==4:
        d = float(diam)/100

    else:
        if len(diam)==3:
            d = float(diam)/10

        else:
            d = float(diam)


    if d >28:
        d *=.1
    
   ### get pitch
    first_letter=0
    for i,l in enumerate(pitch):
        if not l.isdigit():
            first_letter=i
            break

    if first_letter == 0:
        p = int(pitch)

    else:
        p = pitch[:first_letter]


    
####  get extension
    ext = pitch.split(str(p))[1]



     ### format pitch
    if len(str(p))==4:
        p = float(p)/100

    else:
        if len(str(p))==3:
            p = float(p)/10

        else:
            p = float(p)

    if p >22:
        p = float(p)* .1
        p = round(p,8)


    
    return d,p,ext




def get_data_from_APC_prop_file(filename):

    file= open(filename, 'r')

    #print(filename)

    diameter, pitch, extension = get_diam_pitch_ext(filename.split("/")[1])
    #diameter, pitch, extension = get_diam_pitch_ext(filename)


    blocks = []
    lines=[]
    
    for i,line in enumerate(file):
        lines.append(line)
        #print(i,line)
    
        if i == 0:
            name = line.split("(")[0].replace(" ","")
            #print(name)
    
        if "PROP RPM" in line:
            rpm = int(line.split("=")[1])
            blocks.append([i,rpm])
    
    last_line = i


    df = pd.DataFrame()

    for i, block in enumerate(blocks):
        #print(blocks)
        if i == len(blocks)-1:
            #print("last_block")
            data = lines[blocks[i][0]+3:last_line]
    
        else:
            data = lines[blocks[i][0]+3:blocks[i+1][0]-3]
    
    
        
    
        tempfile = open("tempfile.txt","w")
        tempfile.writelines(data)
        tempfile.close()
    
        tdf = pd.read_fwf("tempfile.txt")
        tdf["prop"]=filename.split("/")[1].split("_")[1].split(".")[0]      # extract prop name from filename

        tdf["diam"]=diameter
        tdf["pitch"]=pitch
        tdf["extension"]=extension

        
        tdf["RPM"]=blocks[i][1]
        tdf.columns=["V_mph","J","Pe","Ct","Cp","PWR_Hp","Torque_In_Lbf","Thrust_Lbf","PWR_W","Torque_Nm","Thrust_N","THR/PWR_g_W","Mach","Reyn","FOM","PROP","DIAM","PITCH","EXTENSION","RPM"]
        tdf["V_ms"]= tdf.V_mph * 0.44704
        
        cols = tdf.columns.tolist()
        cols = cols[-6:]+cols[:-6]  
        tdf = tdf[cols]
    
        df = pd.concat([df,tdf])

    try: 
        os.remove("tempfile.txt")
    except:
        pass
        

    return df



### Test the process with one file:

In [24]:
prop_data = path_to_perfiles + "/" + "PER3_12x45MRF-RH.dat"

get_data_from_APC_prop_file(prop_data)

,PROP,DIAM,PITCH,EXTENSION,RPM,V_ms,V_mph,J,Pe,Ct,...,PWR_Hp,Torque_In_Lbf,Thrust_Lbf,PWR_W,Torque_Nm,Thrust_N,THR/PWR_g_W,Mach,Reyn,FOM
0,12x45MRF-RH,12.0,4.5,MRF-RH,1000,0.000000,0.00,0.0000,0.0000,0.0874,...,0.001,0.045,0.058,0.529,0.005,0.257,49.474,0.05,17755.0,0.5813
1,12x45MRF-RH,12.0,4.5,MRF-RH,1000,0.089408,0.20,0.0172,0.0416,0.0855,...,0.001,0.045,0.056,0.529,0.005,0.251,48.461,0.05,17764.0,0.5632
2,12x45MRF-RH,12.0,4.5,MRF-RH,1000,0.174346,0.39,0.0345,0.0815,0.0835,...,0.001,0.045,0.055,0.528,0.005,0.245,47.431,0.05,17774.0,0.5448
3,12x45MRF-RH,12.0,4.5,MRF-RH,1000,0.263754,0.59,0.0517,0.1195,0.0815,...,0.001,0.045,0.054,0.527,0.005,0.239,46.380,0.05,17784.0,0.5262
4,12x45MRF-RH,12.0,4.5,MRF-RH,1000,0.348691,0.78,0.0690,0.1556,0.0793,...,0.001,0.044,0.052,0.525,0.005,0.233,45.310,0.05,17795.0,0.5073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,12x45MRF-RH,12.0,4.5,MRF-RH,20000,46.036179,102.98,0.4531,0.3079,0.0148,...,3.484,10.980,3.907,2598.217,1.241,17.377,0.682,0.96,363569.0,0.0659
26,12x45MRF-RH,12.0,4.5,MRF-RH,20000,47.877984,107.10,0.4712,0.2308,0.0104,...,3.403,10.725,2.751,2537.770,1.212,12.235,0.492,0.96,364090.0,0.0399
27,12x45MRF-RH,12.0,4.5,MRF-RH,20000,49.719789,111.22,0.4894,0.1427,0.0061,...,3.328,10.486,1.601,2481.342,1.185,7.122,0.293,0.96,364625.0,0.0181
28,12x45MRF-RH,12.0,4.5,MRF-RH,20000,51.561594,115.34,0.5075,0.0423,0.0017,...,3.249,10.239,0.447,2422.741,1.157,1.989,0.084,0.96,365166.0,0.0027


### read all files and make a Database

In [25]:
%%time 

DBdf = pd.DataFrame()

for file in glob.glob(path_to_perfiles +"/*.dat"):
    print(file)
    prop_df = get_data_from_APC_prop_file(file)
    DBdf = pd.concat([DBdf,prop_df])
    

10_APC_PERFILES/PER3_1225x375.dat
10_APC_PERFILES/PER3_20x11E.dat
10_APC_PERFILES/PER3_21x12WE.dat
10_APC_PERFILES/PER3_20x15E.dat
10_APC_PERFILES/PER3_12x8.dat
10_APC_PERFILES/PER3_8x10.dat
10_APC_PERFILES/PER3_12x8E.dat
10_APC_PERFILES/PER3_16x55MR.dat
10_APC_PERFILES/PER3_10x10E.dat
10_APC_PERFILES/PER3_8x4E.dat
10_APC_PERFILES/PER3_875x85N.dat
10_APC_PERFILES/PER3_16x6E.dat
10_APC_PERFILES/PER3_17x8.dat
10_APC_PERFILES/PER3_7x6F.dat
10_APC_PERFILES/PER3_10x8E.dat
10_APC_PERFILES/PER3_6x31SF.dat
10_APC_PERFILES/PER3_55x65E.dat
10_APC_PERFILES/PER3_6x3.dat
10_APC_PERFILES/PER3_13x7F.dat
10_APC_PERFILES/PER3_13x45EP.dat
10_APC_PERFILES/PER3_88x85.dat
10_APC_PERFILES/PER3_205x12WE.dat
10_APC_PERFILES/PER3_12x38SF.dat
10_APC_PERFILES/PER3_11x4.dat
10_APC_PERFILES/PER3_27x13E.dat
10_APC_PERFILES/PER3_12x45MRF-RH.dat
10_APC_PERFILES/PER3_7x4F.dat
10_APC_PERFILES/PER3_14x4W.dat
10_APC_PERFILES/PER3_9x45MRF-RH.dat
10_APC_PERFILES/PER3_18x8W.dat
10_APC_PERFILES/PER3_8x375.dat
10_APC_PERFILES

In [26]:
DBdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283830 entries, 0 to 29
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   PROP           283830 non-null  object 
 1   DIAM           283830 non-null  float64
 2   PITCH          283830 non-null  float64
 3   EXTENSION      283830 non-null  object 
 4   RPM            283830 non-null  int64  
 5   V_ms           283830 non-null  float64
 6   V_mph          283830 non-null  float64
 7   J              283830 non-null  float64
 8   Pe             281701 non-null  float64
 9   Ct             281701 non-null  float64
 10  Cp             281701 non-null  float64
 11  PWR_Hp         281701 non-null  float64
 12  Torque_In_Lbf  281701 non-null  float64
 13  Thrust_Lbf     281701 non-null  float64
 14  PWR_W          281701 non-null  float64
 15  Torque_Nm      281701 non-null  float64
 16  Thrust_N       281701 non-null  float64
 17  THR/PWR_g_W    281701 non-null  float6

# Now save the DB as Pandas-DF (pickle) and as CSV file

In [27]:
if True:
    DBdf.to_pickle("APC-Prop-DB.pkl")
    DBdf.to_csv("APC-Prop-DB.csv",index=False,decimal=",")